<a name="about"></a>
About this notebook
======

This notebook assumes you have ran the local Census Regression notebook and you have not deleted the LOCAL_ROOT folder. In this notebook, we will use online and batch prediction on a pre-trained Tensorflow model on CloudML. This notebook will does not assume that the notebook "3. Census Regression Cloud Training" was executed.

<a name="setup"></a>
Setting things up
=====

In [6]:
import datalab_solutions.structured_data as sd

Lets look at the versions of structured_data and TF we have. Make sure TF is 1.0.0, and SD is 0.0.1.

In [7]:
import os
import json
import sys
import numpy as np
import pandas as pd

import tensorflow as tf
from tensorflow.python.lib.io import file_io

import datalab.mlalpha as mlalpha

print('tf ' + str(tf.__version__))
print('sd ' + str(sd.__version__))

tf 1.0.0
sd 0.0.1


This notebook will write files during preprocessing, training, and prediction. Please give a root folder you wish to use.

In [8]:
LOCAL_ROOT = './census_regression_workspace' # This should be the same as what was used in the local census notebook
CLOUD_ROOT = 'gs://' + datalab_project_id() + '-census-regression-datalab'

if not file_io.file_exists(LOCAL_ROOT):
  raise ValueErro('LOCAL_ROOT not found. Did you run the local notebook?')
!gsutil mb {CLOUD_ROOT}

Creating gs://cloud-ml-dev-census-regression-datalab/...
ServiceException: 409 Bucket cloud-ml-dev-census-regression-datalab already exists.


First, let us put the csv files on GCS and the output of preprocessing.

In [5]:
!gsutil -m cp {os.path.join(LOCAL_ROOT, '*_data.csv')} {CLOUD_ROOT}
!gsutil -m cp -r {os.path.join(LOCAL_ROOT, 'training')} {CLOUD_ROOT}

Copying file://./census_regression_workspace/predict_data.csv [Content-Type=text/csv]...
Copying file://./census_regression_workspace/eval_data.csv [Content-Type=text/csv]...
Copying file://./census_regression_workspace/train_data.csv [Content-Type=text/csv]...
/ [3/3 files][200.1 KiB/200.1 KiB] 100% Done                                    
Operation completed over 3 objects/200.1 KiB.                                    
Copying file://./census_regression_workspace/training/model/saved_model.pb [Content-Type=application/octet-stream]...
Copying file://./census_regression_workspace/training/model/variables/variables.data-00000-of-00001 [Content-Type=application/octet-stream]...
Copying file://./census_regression_workspace/training/model/assets.extra/transforms.json [Content-Type=application/json]...
Copying file://./census_regression_workspace/training/evaluation_model/saved_model.pb [Content-Type=application/octet-stream]...
Copying file://./census_regression_workspace/training/evaluat

In [6]:
!gsutil ls {CLOUD_ROOT}/training

gs://cloud-ml-dev-census-regression-datalab/training/evaluation_model/
gs://cloud-ml-dev-census-regression-datalab/training/model/
gs://cloud-ml-dev-census-regression-datalab/training/staging/
gs://cloud-ml-dev-census-regression-datalab/training/train/


<a name="local_preprocessing"></a>
Cloudml Batch Prediction
=====

Batch prediction has two modes. In the 'evaluation' mode, the input data is expected to 100% match the training schema, meaning the target column should exist in the data. In 'prediction' mode, the input data files must match the training schema except that the target column is missing. Note that cloudml batch prediction can be slow on small datasets because it takes a while for a Dataflow job to start.

In [7]:
!gsutil -m rm -r {CLOUD_ROOT}/batch_prediction

CommandException: 1 files/objects could not be removed.


In [8]:
sd.cloud_batch_predict(
  training_ouput_dir=os.path.join(CLOUD_ROOT, 'training'),
  prediction_input_file=os.path.join(CLOUD_ROOT, 'eval_data.*'),
  output_dir=os.path.join(CLOUD_ROOT, 'batch_prediction'),
  mode='evaluation',
  output_format='json'
)


Building package and uploading to gs://cloud-ml-dev-census-regression-datalab/batch_prediction/staging/sd.tar.gz
Starting cloud batch prediction.
Dataflow Job submitted, see Job structured-data-batch-prediction-20170223013027 at https://console.developers.google.com/dataflow?project=cloud-ml-dev


/usr/local/lib/python2.7/dist-packages/apache_beam/coders/typecoders.py:136: UserWarning:

Using fallback coder for typehint: Any.



See above link for job status.


When training is done, {CLOUD_ROOT}/training should contain the folders train, model, evaluation_model, etc.

In [9]:
!gsutil ls  {CLOUD_ROOT}/batch_prediction

gs://cloud-ml-dev-census-regression-datalab/batch_prediction/errors-00000-of-00001.txt
gs://cloud-ml-dev-census-regression-datalab/batch_prediction/predictions-00000-of-00003.json
gs://cloud-ml-dev-census-regression-datalab/batch_prediction/predictions-00001-of-00003.json
gs://cloud-ml-dev-census-regression-datalab/batch_prediction/predictions-00002-of-00003.json
gs://cloud-ml-dev-census-regression-datalab/batch_prediction/staging/
gs://cloud-ml-dev-census-regression-datalab/batch_prediction/tmp/


Cloudml Online Prediction
==========

We first have to deploy an app engine model, and then we can make requests on it.

In [19]:
!gcloud beta ml models create census_model
!gcloud beta ml versions create v1 --model census_model --origin {CLOUD_ROOT}/training/model

Creating version (this might take a few minutes)......done.


In [10]:
sd.cloud_predict(
  model_name='census_model',
  model_version='v1',
  data=['490,64,2,0,1,0,2,8090,015,01,1,00590,00500,1,18,0,2,1',
        '1225,32,5,0,4,5301,2,9680,015,01,1,00100,00100,1,21,2,1,1',
        '1226,30,1,0,1,0,2,8680,020,01,1,00100,00100,1,16,0,2,1']
)

,key_from_input,predicted_target
0,490,32.4215
1,1225,52.2748
2,1226,23.2904


In [9]:
sd.cloud_predict(
  model_name='census_model',
  model_version='v1',
  data=pd.DataFrame(
    [[490,64,2,0,1,0,2,8090,"015","01",1,"00590","00500",1,18,0,2,1],
     [1225,32,5,0,4,5301,2,9680,"015","01",1,"00100","00100",1,21,2,1,1],
     [1226,30,1,0,1,0,2,8680,"020","01",1,"00100","00100",1,16,0,2,1]])
)

,key_from_input,predicted_target
0,490,32.4215
1,1225,52.2748
2,1226,23.2904


Cleaning things up
=====

If you want to delete the files you made on GCS, uncomment and run the next cell.

In [10]:
#!gsutil rm -fr {CLOUD_ROOT}

To delete the models created, uncomment and run the next cell.

In [13]:
#!gcloud beta ml versions delete v1 --model census_model 
#!gcloud beta ml models delete census_model 

Deleting version [v1]......done.
Deleting model [census_model]...done.
